In [34]:
import pandas as pd
import numpy as np
import sqlite3
import os

## creating a specific Dataframe recipe => ingredient (bool)


In [35]:
path = os.path.realpath('../Data')
print(path)

con = sqlite3.connect(path+'/RecipeDB.db')
cur = con.cursor()

ing_df = pd.read_sql('SELECT familyID FROM IngFamily ', con)
recs_df = pd.read_sql('SELECT recID From Recipe', con)

index = [x for x in recs_df['recID']]

data = np.zeros((len(index), len(ing_df)), dtype= np.bool_)
df = pd.DataFrame(data = data, index= index, columns= ing_df["familyID"])
df.describe()

C:\Users\whyno\PycharmProjects\RecipeRecommender\Resources\Data


familyID,58877cf3ed71a76e9c03fb31,58877cf3ed71a76e9c03fb33,58877cf3ed71a76e9c03fb34,58877cf3ed71a76e9c03fb35,58877cf3ed71a76e9c03fb37,58877cf3ed71a76e9c03fb38,58877cf3ed71a76e9c03fb39,58877cf3ed71a76e9c03fb3a,58877cf3ed71a76e9c03fb3b,58877cf3ed71a76e9c03fb3c,...,58877cf3ed71a76e9c03fc54,58877cf3ed71a76e9c03fc57,58877cf3ed71a76e9c03fc58,58877cf3ed71a76e9c03fc59,58877cf3ed71a76e9c03fc5a,58cba71cc6243b4863443ce3,5901b0d4c6243b746b6221f2,5901b0d4c6243b746b6221f7,5901b0d4c6243b746b6221f9,5901b0d4c6243b746b6221fb
count,1808,1808,1808,1808,1808,1808,1808,1808,1808,1808,...,1808,1808,1808,1808,1808,1808,1808,1808,1808,1808
unique,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,1808,1808,1808,1808,1808,1808,1808,1808,1808,1808,...,1808,1808,1808,1808,1808,1808,1808,1808,1808,1808


In [36]:
print(df.shape)

(1808, 224)


In [38]:
for ing in cur.execute("SELECT familyID, recID FROM IngInRec NATURAL JOIN Ingredients ;").fetchall():
    # print(ing[1])
    df.at[ing[1], ing[0]] = True
    # print(df.loc[ing[1]][ing[0]])
    

In [39]:
df.describe()



familyID,58877cf3ed71a76e9c03fb31,58877cf3ed71a76e9c03fb33,58877cf3ed71a76e9c03fb34,58877cf3ed71a76e9c03fb35,58877cf3ed71a76e9c03fb37,58877cf3ed71a76e9c03fb38,58877cf3ed71a76e9c03fb39,58877cf3ed71a76e9c03fb3a,58877cf3ed71a76e9c03fb3b,58877cf3ed71a76e9c03fb3c,...,58877cf3ed71a76e9c03fc54,58877cf3ed71a76e9c03fc57,58877cf3ed71a76e9c03fc58,58877cf3ed71a76e9c03fc59,58877cf3ed71a76e9c03fc5a,58cba71cc6243b4863443ce3,5901b0d4c6243b746b6221f2,5901b0d4c6243b746b6221f7,5901b0d4c6243b746b6221f9,5901b0d4c6243b746b6221fb
count,1808,1808,1808,1808,1808,1808,1808,1808,1808,1808,...,1808,1808,1808,1808,1808,1808,1808,1808,1808,1808
unique,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,1,2
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,1763,1805,1807,1756,1802,1759,1802,1798,1733,1716,...,1800,1562,1545,1806,1807,1805,1799,1802,1808,1804


## Creating dynamic ingredient-questions

In [134]:
test = [(str(i),df[df[i]==True].shape[0]/1808) for i in df.columns]


def closest(lst, K):  
    l = np.array([i[1] for i in lst])
    # print(l) 
    idx = (np.abs(l - K)).argmin()
    return lst[idx]

def dynamic(df, l = 5):
    for i in range(l):
        size = df.shape[0]
        percentage_list = [(str(i),df[df[i]==True].shape[0]/size) for i in df.columns]
        temp = closest(percentage_list,0.5)
        print("Magst du",cur.execute("Select familyName from IngFamily where familyID = ?", (temp[0],)).fetchone()[0]," ?")
        # some input = True , no input = False
        user =bool(input()) # to google api
        # here we need to save result into user class
        df = df[df[temp[0]] == user]


dynamic(df, l = 5)



Magst du Zwiebel  ?


Magst du Knoblauch  ?


 w


Magst du Käse  ?


Magst du Frühlingszwiebel  ?


Magst du Kartoffeln  ?
